In [ ]:
import json

import matplotlib.pyplot as plt
from utils import get_linestyle_dict, total_size

from soft_label_learning.config import path_output, path_repository
from soft_label_learning.experiments.experiment_settings import (
    methods_without_threshold,
    q2_settings,
)
from soft_label_learning.experiments.process_synthetic_data import (
    get_noise_replace_dict,
    plot_variation_over_sets_on_axis,
)

Obtaining the experiment base parameters

In [ ]:
# TODO set datetime to the desired result folder
settings_dict, result_path = (
    q2_settings,
    "q2_results_processed_date_hh_mm_ss",
)

In [ ]:
methods = methods_without_threshold
datasets = [x[:-4] for x in settings_dict["dataset"]]
classifiers = [x for x in settings_dict["clf"]]
metrics = ["TVD", "hard_soft_AUC"]
fixed_settings = settings_dict.copy()

fixed_settings["ens_propagation"] = True
fixed_settings["alpha"] = 1

#### Loading results

In [ ]:
with open(
    path_repository / "experiments" / "synthetic_data_results" / f"{result_path}.json",
    "r",
) as f:
    result_dict_load_json = json.load(f)

In [ ]:
result_dict = result_dict_load_json
print(f"total size of the result dict: {total_size(result_dict)}")

#### Generate Figures

In [ ]:
save = True
base_norm = False

adjusted_methods = methods

method_style_dict = get_linestyle_dict()

save_path = path_output / "Q2"

noise_replace_dict = get_noise_replace_dict()

for metric in ["hard_soft_AUC", "TVD"]:
    for mtvd in ["1", "2"]:
        # Create a figure with a 6x4 grid of subplots
        fig, axes = plt.subplots(6, 4, figsize=(15, 20))

        # Iterate over the 6x4 grid and plot something in each subplot
        for col_idx, classifier in enumerate(classifiers):
            for row_idx, noise_type in enumerate(settings_dict["noise_type"][1:]):
                ax = axes[row_idx, col_idx]

                plot_variation_over_sets_on_axis(
                    ax,
                    result_dict["rf"][mtvd],
                    datasets=datasets,
                    title=f"{classifier}, {noise_replace_dict[noise_type]}",
                    linestyle_dict=method_style_dict,
                    metric=metric,
                    clf=classifier,
                    noise_type=noise_type,
                    method=adjusted_methods,
                    base_norm=base_norm,
                )

        # Extract the handles and labels from one of the axes
        handles, labels = axes[0, 0].get_legend_handles_labels()

        # Create a legend for the entire figure at the bottom
        fig.legend(
            handles, labels, loc="upper center", bbox_to_anchor=(0.51, -0.00), ncol=7
        )

        metric_dict = {"hard_soft_AUC": "AUC", "TVD": r"$\overline{TVD}$"}

        for i in range(6):
            axes[i, 0].set_ylabel(metric_dict[metric])

        for i in range(4):
            axes[5, i].set_xlabel("Noise level")

        # Adjust layout
        plt.tight_layout()
        plt.show()

        if save:
            fig.savefig(
                bbox_inches="tight",
                fname=save_path / f"RF_{mtvd}_{metric}_base_{base_norm}.png",
                dpi=300,
            )